<a href="https://colab.research.google.com/github/Kimhansav/everynocode_search_engine/blob/main/BP_make_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#KcBERT 학습용 데이터셋을 제작하는 코드(로컬)
#제작할 데이터셋은 pretraining용 데이터셋(soynlp 학습용 데이터), finetuning용 데이터셋(next sentence prediction)

In [1]:
!pip install kss
!pip install datasets
!pip install soynlp
import pandas as pd
import numpy as np
import re
import os
import torch
import tensorflow as tf
import urllib.request
from soynlp.normalizer import *
from kss import Kss
from tqdm import tqdm
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import AutoTokenizer, shape_list, TFBertModel, RobertaTokenizerFast, RobertaForSequenceClassification, TextClassificationPipeline, pipeline, BertTokenizer, BertForNextSentencePrediction,  TrainingArguments
from datasets import Dataset, load_dataset
from sklearn.model_selection import StratifiedKFold, train_test_split
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
#전처리된 파일 로드(BP_spacing 건너뛰기)
talk_save_path = '/content/drive/My Drive/talk_preprocess_result_short.xlsx'
qna_path = '/content/drive/My Drive/community_qna_preprocessed.xlsx'
all_contents_path = '/content/drive/My Drive/community_all_contents_preprocessed.xlsx'
all_comments_path = '/content/drive/My Drive/community_all_comments_preprocessed.xlsx'

df_talk = pd.read_excel(talk_save_path)
df_qna = pd.read_excel(qna_path)
df_all_contents = pd.read_excel(all_contents_path)
df_all_comments = pd.read_excel(all_comments_path)

In [3]:
qna_path = '/content/drive/My Drive/community_qna_preprocessed.xlsx'
df_qna = pd.read_excel(qna_path)

In [4]:
#직접 제작한 카카오톡 질문답변 데이터셋 로드
talk_finetune_path = '/content/drive/My Drive/talk_finetune_dataset.xlsx'

df_talk_finetune = pd.read_excel(talk_finetune_path, index_col = 0)

#Soyspacing 학습시킨 후에 띄어쓰기 교정

In [7]:
#일단 보류

#Pretrain 데이터셋, 제공받은 모든 데이터 사용


In [7]:
split_sentences = Kss('split_sentences')

In [8]:
#kss로 리스트 안에 있는 여러 문장으로 이루어진 글을 문장 단위로 분리하는 함수
def separate_to_sentences_and_deduplicate(DataFrame, columns):
    sentences_list = []
    for column in columns: #데이터프레임의 각 열에 대해서 처리
        text_list = DataFrame[column].to_list()
        sentences_list.append(split_sentences(text_list))
    return list(set(sum([lists for lists in sentences_list[0]], [])))

In [10]:
#데이터프레임의 열들을 리스트로 변환
talk_list = separate_to_sentences_and_deduplicate(df_talk, ['text'])


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [12]:
#데이터프레임의 NaN 대신 '' 채워넣기
df_qna.fillna('', inplace = True)
df_all_contents.fillna('', inplace = True)
df_all_comments.fillna('', inplace = True)

In [11]:
qna_list = separate_to_sentences_and_deduplicate(df_qna, ['내용', '답변', '댓글'])

In [12]:
all_contents_list = separate_to_sentences_and_deduplicate(df_all_contents, ['내용', '답변', '댓글'])

In [13]:
all_comments_list = separate_to_sentences_and_deduplicate(df_all_comments, ['내용', '대댓글', '댓글'])
print(all_comments_list)

['감사드려요 !', '결제 페이지에서 똑같은 명령값으로 총액과 주문 리스트를 누적하는 값을 기재했는데 제 개인 앱에서는 연동이 되고 메인앱에서는 연동이 안되는 현상입니다!', '컨설팅 또 해줘', '찬찬히 씹고 뜯어 보겠습니다.', '희철님이 말씀하신대로 고도화 된 개인화 추천은 딥한 기술의 영역이기 때문에ㅠ 아직 버블 단독으로는 살짝 어려움이 있겠네요!^^', '화이팅!', '길게 적었다가 올리신 글을 다시 보고 혹시나 해서 여쭙는데요.', '근데 더 자세하고 정리가 잘된 글이네요', '저도 궁금하내요', 'DB부터 구조를 짜고있는데 이것저것 틀부터 잡아보면서 계속 질문글 올려볼게요!', '네 인프런에도 무료 강의가 있고 자체적으로 블로그에서도 무료 강의가 있습니다.', '너무 기대됩니다!', '유튜브 튜토리얼인데, 거의 생각하시는 기능이랑 비슷하네요.', '민교님! ㅎ', '찬우님, 최고예요!', '현재 내용 만으로는 파악이 어렵네요 ㅠ', '해보고 결과 남겨드리겠습니다!', '레고 하듯이 어떻게든 뚝딱뚝딱 하다보면 개발 단계에서는 어찌어찌 돌아가니까 말이죠.', '버블로 프로젝트를 실행하다니 대단하십니다.', '제가 직접 해야할까봐요 ㅎㅎㅎ', '그러나 버블 기본 타입의 값들은 중복으로 들어가지 않는 것 같습니다.', '이번주 너무 정신이 없어서 가시기 전에 연락드리려다가 결국 못하고 말았네요 ㅠㅠ', '충분한 확장 가능성이 있습니다~!', '저도 저 사이트 들어가서 봤는데 재은님과 거의 같은 생각입니다.', '제가 직접 부딪혀서 파악했으면 더 시간이 걸렸을 이런 귀중한 정보를 공유해주셔서 감사드립니다', '열심히 배워갈게요', '친절한 답변 너무 감사드립니다', '앗 피드백 감사합니다!', '큼지막하게 나눠보자면원페이지 앱으로 페이지 구조 변경 -> 페이지 이동 속도가 훨씬 빨라집니다.', '그런데 제가 옵션셋을 사용안한 이유는 회사마다 직접 설정할 수 있게 해줄려고 한거였거든요.', '리유저블 엘리먼트는 웹앱전반에서 반복적으로 사용해야 할 필요가 있

In [14]:
#하나의 리스트로 결합
result_list = talk_list + qna_list + all_contents_list + all_comments_list
print(result_list[:10])
print(len(result_list))
df_result = pd.DataFrame(result_list, columns = ['text'])
print(df_result)
print(len(df_result))

['저도 나중에 시간되면 자세히 공유하겠습니다.', '새해 복 많이 받으세요', '이전버전으로 가서 디플로이를 하려면 live버전과 sync를 맞추라고 하는 알림이 뜨고 이걸 누르면 merge되면서 현재 에러난 상태와 다를바 없게 되는 것 같습니다..', '아직은 초기 버전이기 때문에 일반적인 이커머스 제품에 커뮤니티 방 기능 밖에 추가되지 않았지만, 게이밍피케이션을 적극 활용할 예정입니다.', '이 방법 시도해보셨을까요?', '같은 스타일을 여기저기 그룹에 지정해줄때 하나 만들어두고 여기저기에 한번에 활용하는거죠!', '다들', '컨디션으로 안돼서 찾아보고 있었는데 감사합니다', 'API connector로 연동하다 생긴 아래 오류 해결점을 아시는분은 도움 주시면 감사하겠습니다!', 'PAD와 같은 RPA도구를 이용하시면  됩니다.']
29185
                                                    text
0                               저도 나중에 시간되면 자세히 공유하겠습니다.
1                                           새해 복 많이 받으세요
2      이전버전으로 가서 디플로이를 하려면 live버전과 sync를 맞추라고 하는 알림이 ...
3      아직은 초기 버전이기 때문에 일반적인 이커머스 제품에 커뮤니티 방 기능 밖에 추가되...
4                                         이 방법 시도해보셨을까요?
...                                                  ...
29180                              노코드 빌딩의 정수를 경험하셨네요 ㅎㅎ
29181                                         답변 감사드립니다.
29182                                           응원하겠습니다.
29183 

In [15]:
#전체 데이터에 대해 soynlp.normalize 적용?
#일단 보류

In [16]:
#.csv 파일로 저장(.xlsx로 저장 시 illegalcharactererror 발생)
pretrain_data_save_path = '/content/drive/My Drive/KcBERT_pretrain_dataset.csv'

df_result.to_csv(pretrain_data_save_path)

#Finetune 데이터셋


In [24]:
#slug가 있다면 ['내용'] == 질문
df_question_answer = df_qna[pd.isna(df_qna['Slug']) == False]
df_question_answer = df_question_answer[pd.isna(df_question_answer['답변']) == False]

print(df_question_answer)
print(len(df_question_answer))

print(df_question_answer.iloc[0])
print(df_question_answer.iloc[1]['답변'])

     Unnamed: 0                                                 내용  \
0             0  안녕하세요! 우선 모두의노코드 사이트 런칭을 축하드립니다.이렇게 첫번째 질문 글을 ...   
1             1  저도 궁금해져서 열심히 찾아봤는데 콜아웃 같은 블록을 넣는 명령어나 함수는 찾지 못...   
2             2  안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...   
3             3  헤더그룹 안에 로고와 CTA를 넣으시고 max width를 지정하시면 될 것 같아요...   
4             4  안녕하세요.제주도에서 노션으로 동네 친구 매칭 서비스를 만들고 있습니다.어제 웨비나...   
..          ...                                                ...   
573         573  정보 구조나 와이어 프레임을 그릴 때 보통 피그마를 쓰시는 분이 많은데 한글로 된 ...   
574         574                                                      
575         575  안녕하세요! 현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하려고 해...   
576         576  안녕하세요! 버블로 외주개발하고 있는 리트머스팀입니다.yongjun@cigro.io...   
577         577                 안녕하세요~아래 링크로 톡 남겨놓으시면 견적 도와드리겠습니다!   

                                                    답변  \
0    저도 궁금해져서 열심히 찾아봤는데 콜아웃 같은 블록을 넣는 명령어나 함수는 찾지 못...   
1                                          

질문 + 답변에다가 답변에 달린 댓글들까지 입력 데이터로 처리할 경우 퀄리티가 더 좋아질 수 있지만, max_length를 초과하지 않는 방향에 따라 답변 데이터만 이용, 나중에 개선방안 찾아보기

community_qna_preprocessed 데이터에서 이미지 파일을 base64 인코딩한 텍스트가 섞여 있는데, 데이터 파일 추출 과정에서 직접 제거하는 게 좋을 것 같음.

In [27]:
#커뮤니티 파일을 가공해서 finetune 데이터셋을 제작
finetune_dict = {}

#slug가 있다면 ['내용'] == 질문
df_question_answer = df_qna[df_qna['Slug'] != '']
df_question_answer = df_question_answer[df_question_answer['답변'] != '']

print(df_question_answer.iloc[2])

#['답변']의 텍스트를 ' , ' 로 구분 후 질문에 하나씩 쌓기
sent1_list, sent2_list, label_list, index_list = [], [], [], []
for i in range(len(df_question_answer)):
  row = df_question_answer.iloc[i]
  question_text = row['내용']
  answer_text_list = row['답변'].split(' , ')
  for j in range(len(answer_text_list)):
    if answer_text_list[j] != '':
      sent1_list.append(question_text + ''.join(answer_text_list[:j]))
      sent2_list.append(answer_text_list[j])
      label_list.append(1)
      index_list.append(i)

finetune_dict = {'sent1' : sent1_list, 'sent2' : sent2_list, 'label' : label_list, 'sent2_index' : index_list}

df_community_finetune = pd.DataFrame.from_dict(finetune_dict)
print(df_community_finetune)

Unnamed: 0                                                    4
내용            안녕하세요.제주도에서 노션으로 동네 친구 매칭 서비스를 만들고 있습니다.어제 웨비나...
답변            충분히 버블로 개발가능할 것 같아요! 이미 비슷한 수준의 어플을 개발한 사례가 많아...
댓글            버블은 다 되요. 개인별 능력이 좌우합니다. 치킨은 살 안찌듯. , 1:1 매칭하는...
관련 링크                                                          
작성일                                         2023-01-12 19:01:00
작성자                                                         김정민
제목                                         버블로 소개팅 앱 만들 수 있을까요?
첨부 파일                                                          
Slug                                        버블로-소개팅-앱-만들-수-있을까요
게시글 링크                                                         
Name: 4, dtype: object
                                                 sent1  \
0    안녕하세요! 우선 모두의노코드 사이트 런칭을 축하드립니다.이렇게 첫번째 질문 글을 ...   
1    안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...   
2    안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...   
3    안녕하세요.제주도에서 노션으로 동네 친구 매칭 서비스를 만들고 있

In [28]:
#파인튜닝 데이터셋에서 n배의 negative sample을 생성하는 코드
def create_negative_sample(DataFrame, n):
  DataFrame_copy = DataFrame.copy() #원본 데이터셋을 변화시키지 않기 위해 deep copy
  length = len(DataFrame_copy)
  for i in range(length):
    row = DataFrame_copy.iloc[i]
    negative_candidate_ids = list(set(DataFrame_copy['sent2_index'].tolist())) #.remove(row['sent2_index']) #해당 행의 sent2를 negative sample 후보군에서 제외
    negative_candidate_ids.remove(row['sent2_index'])
    negative_sample_ids = list(np.random.choice(negative_candidate_ids, n, replace = False))
    for idx in negative_sample_ids:
      new_row = pd.Series({'sent1' : row['sent1'], 'sent2' : np.random.choice(DataFrame_copy[DataFrame_copy['sent2_index'] == idx]['sent2'].tolist(), 1)[0], 'label' : 0, 'sent2_index' : idx}) #카톡 데이터는 idx 중복이 없기 때문에 랜덤추출로 1개를 뽑아도 달라지는 건 없고, 커뮤니티 데이터의 경우 idx 중복이 있기 때문에 해당 idx를 가진 sent2 중 하나를 뽑아야 함.
      DataFrame_copy = pd.concat([DataFrame_copy, new_row.to_frame().T], ignore_index = True)
  return DataFrame_copy

In [29]:
#커뮤니티 파인튜닝 데이터셋에서 negative sample 생성
df_community_finetune_negative_sampled = create_negative_sample(df_community_finetune, 7)
# df_community_finetune_negative_sampled = df_community_finetune_negative_sampled.drop('sent2_index', axis = 1)
print(len(df_community_finetune_negative_sampled))
print(df_community_finetune_negative_sampled)

1824
                                                  sent1  \
0     안녕하세요! 우선 모두의노코드 사이트 런칭을 축하드립니다.이렇게 첫번째 질문 글을 ...   
1     안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...   
2     안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...   
3     안녕하세요.제주도에서 노션으로 동네 친구 매칭 서비스를 만들고 있습니다.어제 웨비나...   
4     모두의노코드 만드신 분들께 질문 드리고 싶어요.제가 버블로 만든 서비스 관련해 모바...   
...                                                 ...   
1819  안녕하세요! 현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하려고 해...   
1820  안녕하세요! 현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하려고 해...   
1821  안녕하세요! 현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하려고 해...   
1822  안녕하세요! 현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하려고 해...   
1823  안녕하세요! 현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하려고 해...   

                                                  sent2 label sent2_index  
0     저도 궁금해져서 열심히 찾아봤는데 콜아웃 같은 블록을 넣는 명령어나 함수는 찾지 못...     1           0  
1     헤더그룹 안에 로고와 CTA를 넣으시고 max width를 지정하시면 될 것 같아요...     1           1  
2     모두의노코드를 여쭤보신 것인지 모르겠어서 이게 맞는지는 모르겠습니다.모두의노코드의 ...   

In [30]:
print(df_community_finetune_negative_sampled[df_community_finetune_negative_sampled['sent2'] == ''])
print(df_community_finetune[df_community_finetune['sent2_index'] == 13])

Empty DataFrame
Columns: [sent1, sent2, label, sent2_index]
Index: []
                                                sent1  \
15  플러스 버튼 (+) 을 클릭하면 리피팅 그룹 내의 로우가 +1 되도록 리피팅 그룹의...   
16  플러스 버튼 (+) 을 클릭하면 리피팅 그룹 내의 로우가 +1 되도록 리피팅 그룹의...   

                                                sent2  label  sent2_index  
15  일단 제가 소영님의 질문을 정확히 이해했는지 잘 모르겠습니다. 질문대로라면 리피팅그...      1           13  
16  저는 플러그인 요청하셔서 간단하게 제가 변형해서 쓰는 템플릿 살포시 투척 해놓고 가...      1           13  


In [31]:
print(df_talk_finetune)

                                                 sent1  \
0     deltaPercent의 절대값이 큰 값부터 데이터를 정렬하고 싶은데.deltaP...   
1     유저의 input url에 따른 rss 피드 긁어오기를 하고 싶은데 헷갈리는 부분...   
2     혹시 버블 워크로드 최적화하는 팁이 있을까요? 아직 레거시 요금제 쓰고 있긴 한데...   
3     혹시 버블 워크로드 최적화하는 팁이 있을까요? 아직 레거시 요금제 쓰고 있긴 한데...   
4     혹시 어떤 product 이신가요? 와...이러면 버블 못 쓸 거 같은데요 ㅡㅡ;;;;   
..                                                 ...   
496  각 element에다가 condition을 따로 쓰면 workflow에 안들어가있기...   
497  각 element에다가 condition을 따로 쓰면 workflow에 안들어가있기...   
498   안녕하세요, 혹시 버블에서 three.js라는 라이브러리 연결을 도와주실 분 계실...   
499   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   
500   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   

                                                 sent2  label  sent2_index  
0     익스프레션 내에서 조건식을 사용할 수가 없어서 condition을 사용하는 수밖에...      1            1  
1     이게 근데 한번 락인되니까 DB 이관한다는 선택지를 고르기가 참 어렵네요.. 또 ...      0           13  
2     1. Do a search for 할 때 constraints 걸어서 검색량 줄이...      1           

In [32]:
#카카오톡 파인튜닝 데이터셋에서 negative sample 생성
df_talk_finetune_negative_sampled = create_negative_sample(df_talk_finetune, 7)
df_talk_finetune_negative_sampled = df_talk_finetune_negative_sampled.drop('sent2_index', axis = 1)
print(len(df_talk_finetune_negative_sampled))
print(df_talk_finetune_negative_sampled)

4000
                                                  sent1  \
0      deltaPercent의 절대값이 큰 값부터 데이터를 정렬하고 싶은데.deltaP...   
1      유저의 input url에 따른 rss 피드 긁어오기를 하고 싶은데 헷갈리는 부분...   
2      혹시 버블 워크로드 최적화하는 팁이 있을까요? 아직 레거시 요금제 쓰고 있긴 한데...   
3      혹시 버블 워크로드 최적화하는 팁이 있을까요? 아직 레거시 요금제 쓰고 있긴 한데...   
4      혹시 어떤 product 이신가요? 와...이러면 버블 못 쓸 거 같은데요 ㅡㅡ;;;;   
...                                                 ...   
3995   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   
3996   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   
3997   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   
3998   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   
3999   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   

                                                  sent2 label  
0      익스프레션 내에서 조건식을 사용할 수가 없어서 condition을 사용하는 수밖에...     1  
1      이게 근데 한번 락인되니까 DB 이관한다는 선택지를 고르기가 참 어렵네요.. 또 ...     0  
2      1. Do a search for 할 때 constraints 걸어서 검색량 줄이...     1  
3                    Do a sear

In [33]:
print(df_talk_finetune_negative_sampled[df_talk_finetune_negative_sampled['sent2'] == ''])

Empty DataFrame
Columns: [sent1, sent2, label]
Index: []


In [34]:
#두 파인튜닝 데이터셋을 하나로 합치기
df_KcBERT_finetune = pd.concat([df_community_finetune_negative_sampled, df_talk_finetune_negative_sampled], ignore_index = True)
print(len(df_KcBERT_finetune))

5824


In [35]:
#.xlsx 파일로 저장
finetune_data_save_path = '/content/drive/My Drive/KcBERT_finetune_dataset.xlsx'

df_KcBERT_finetune.to_excel(finetune_data_save_path)